In [15]:
import pandas as pd
import numpy as np
import streamlit as st
from fastapi import FastAPI
import uvicorn

In [30]:
games=pd.read_parquet('Datasets/steam_games.parquet')
users_items=pd.read_parquet('Datasets/users_items.parquet')
user_reviews=pd.read_parquet('Datasets/user_reviews.parquet')

In [33]:
games.developer.count


<bound method Series.count of 88310                    Kotoshiro
88311             Secret Level SRL
88312                 Poolians.com
88313                         彼岸领域
88314                         None
                    ...           
120440          Nikita "Ghost_RUS"
120441                      Sacada
120442    Laush Dmitriy Sergeevich
120443               xropi,stev3ns
120444                        None
Name: developer, Length: 32135, dtype: object>

In [34]:
user_reviews

,user_id,item_id,helpful,recommend,review,preprocessed_review,sentiment
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,simple yet with great replayability. in my opi...,1
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,it's unique and worth a playthrough.,1
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,great atmosphere. the gunplay can be a bit chu...,1
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,i know what you think when you see this title ...,1
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,for a simple (it's actually not all that simpl...,1
...,...,...,...,...,...,...,...
59328,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,a must have classic from steam definitely wort...,1
59329,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,this game is a perfect remake of the original ...,1
59330,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,had so much fun plaing this and collecting res...,1
59331,LydiaMorley,730,No ratings yet,True,:D,:d,1


In [13]:
users_items

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
1,76561197970982479,20,0.0
2,76561197970982479,30,7.0
3,76561197970982479,40,0.0
4,76561197970982479,50,0.0
...,...,...,...
5170010,76561198329548331,373330,0.0
5170011,76561198329548331,388490,3.0
5170012,76561198329548331,521570,4.0
5170013,76561198329548331,519140,3.0


Pruebo aca el mas facil

In [62]:
games = games.dropna(subset=['id'])
user_reviews = user_reviews.dropna(subset=['item_id'])


In [60]:
games['id'].isnull().sum()

0

In [63]:
games['id']=games['id'].astype(dtype='int64')
user_reviews['item_id']=user_reviews['item_id'].astype(dtype='int64')

C:\Users\juan\AppData\Local\Temp\ipykernel_29848\3798767444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_reviews['item_id']=user_reviews['item_id'].astype(dtype='int64')


In [66]:
games['id']
user_reviews['item_id']

0          1250
1         22200
2         43110
3        251610
4        227300
          ...  
59328        70
59329    362890
59330    273110
59331       730
59332       440
Name: item_id, Length: 59305, dtype: int64

In [69]:
df_query=pd.merge(games,user_reviews,left_on='id',right_on='item_id')


In [113]:

respuesta=df_query[df_query['developer']=='Valve'].sentiment.value_counts()

In [118]:
respuesta[0]


116

In [117]:
respuesta

sentiment
1    8633
2     948
0     116
Name: count, dtype: int64